Calculate the Call/Option price of a European Option, using Risk-Neutral Monte Carlo simulations

In [1]:
# importing necessary packages
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import statistics as stat
import math as m

In [2]:
def geometric_BM_RW(S0, r, T, vol):
    # geometric_BM_RW: Simulates the terminal stock price using Geometric Brownian Motion
    # S0: Initial stock price
    # r: Risk-free interest rate
    # T: Time to maturity (in years)
    # vol: Volatility of the stock
    
    # Generate a random sample from a standard normal distribution and scale it by sqrt(T)
    W = np.random.standard_normal(size=1) * np.sqrt(T)
    
    # Calculate the exponent for the stock price using risk-neutral drift and volatility
    X = (r - 0.5 * vol**2) * T + vol * W
    
    # Calculate the terminal stock price using the exponential of X
    S = S0 * np.exp(X)
    return S  # Return the simulated terminal stock price


def sample_mean(data):
    # sample_mean: Calculates the mean of the provided data
    # data: A list of values to compute the average
    
    sum = 0  # Initialize sum variable
    for i in data:  # Loop through each element in data
        sum += i  # Add each element to the sum
    average = sum / len(data)  # Calculate the average by dividing the sum by the number of elements
    return average  # Return the average


def call_payoff(ST, K):
    # call_payoff: Calculates the payoff of a European call option at maturity
    # ST: Terminal stock price
    # K: Strike price
    
    if ST > K:  # If terminal stock price is greater than strike price
        terminal_pay = ST - K  # Payoff is the difference between ST and K
    else:
        terminal_pay = 0  # Otherwise, the payoff is 0 (option expires worthless)
    return terminal_pay  # Return the payoff


def put_payoff(ST, K):
    # put_payoff: Calculates the payoff of a European put option at maturity
    # ST: Terminal stock price
    # K: Strike price
    
    if K > ST:  # If strike price is greater than terminal stock price
        terminal_pay = K - ST  # Payoff is the difference between K and ST
    else:
        terminal_pay = 0  # Otherwise, the payoff is 0 (option expires worthless)
    return terminal_pay  # Return the payoff


def MC_simulation_call(S0, K, r, T, vol, simulations):
    # MC_simulation_call: Simulates the price of a European call option using Monte Carlo
    # S0: Initial stock price
    # K: Strike price
    # r: Risk-free interest rate
    # T: Time to maturity (in years)
    # vol: Volatility of the stock
    # simulations: Number of Monte Carlo simulations
    
    terminal_list = []  # Initialize a list to store the payoffs for each simulation
    
    # Run Monte Carlo simulations
    for i in range(1, simulations + 1):
        # Simulate the terminal stock price using Geometric Brownian Motion
        ST = geometric_BM_RW(S0, r, T, vol).tolist()[-1]
        
        # Calculate the payoff for a call option at maturity
        terminal_payoff = call_payoff(ST, K)
        
        # Append the payoff to the terminal_list
        terminal_list.append(terminal_payoff)
    
    # Calculate the average payoff across all simulations
    average_terminal_payoff = sample_mean(terminal_list)
    
    # Discount the average payoff back to the present value using the risk-free rate
    discounted_payoff = average_terminal_payoff * m.exp(-r * T)
    
    return discounted_payoff  # Return the estimated call option price


def MC_simulation_put(S0, K, r, T, vol, simulations):
    # MC_simulation_put: Simulates the price of a European put option using Monte Carlo
    # S0: Initial stock price
    # K: Strike price
    # r: Risk-free interest rate
    # T: Time to maturity (in years)
    # vol: Volatility of the stock
    # simulations: Number of Monte Carlo simulations
    
    terminal_list = []  # Initialize a list to store the payoffs for each simulation
    
    # Run Monte Carlo simulations
    for i in range(1, simulations + 1):
        # Simulate the terminal stock price using Geometric Brownian Motion
        ST = geometric_BM_RW(S0, r, T, vol).tolist()[-1]
        
        # Calculate the payoff for a put option at maturity
        terminal_payoff = put_payoff(ST, K)
        
        # Append the payoff to the terminal_list
        terminal_list.append(terminal_payoff)
    
    # Calculate the average payoff across all simulations
    average_terminal_payoff = sample_mean(terminal_list)
    
    # Discount the average payoff back to the present value using the risk-free rate
    discounted_payoff = average_terminal_payoff * m.exp(-r * T)
    
    return discounted_payoff  # Return the estimated put option price


In [12]:
MC_C = MC_simulation_call(29, 28, 0.05, 3/12, 0.15, 100000)
print("European Call Option Price: £", MC_C)

MC_P = MC_simulation_put(9, 10, 0.04, 2, 0.1, 100000)
print("European Put Option Price: £", MC_P)

European Call Option Price: £ 1.6866860666776358
European Put Option Price: £ 0.6373575474221568


Alternative Risk-Neutral Monte Carlo approach, including number of steps in the random walk leading up to the terminal stock price (increasing number of steps decreases accuracy and precision of result due to increasing variance in the Brownian motion over time)

In [4]:
def geometric_BM_RW_steps(S0, r, T, N, vol):
    # geometric_BM_RW_steps: Simulates the stock price evolution using Geometric Brownian Motion with multiple steps
    # S0: Initial stock price
    # r: Risk-free interest rate
    # T: Time to maturity (in years)
    # N: Number of discrete time steps in the random walk
    # vol: Volatility of the stock (standard deviation of stock returns)
    
    t = np.linspace(0, T, N+1)  # Create an array of time steps from 0 to T, with N+1 points (N intervals)
    
    # Generate N+1 standard normal random variables (one for each time step) and scale them by sqrt(t)
    # This simulates the Brownian motion over N+1 time intervals
    W = np.random.standard_normal(size=N+1) * np.sqrt(t)
    
    # Calculate the stock price at each time step using the GBM model
    # The stock price follows the stochastic differential equation for GBM
    # X = (r - 0.5 * vol^2) * t + vol * W
    X = (r - 0.5 * vol**2) * t + vol * W
    
    # Calculate the stock price S at each time step by exponentiating X and scaling by the initial price S0
    S = S0 * np.exp(X)
    
    return S  # Return the array of simulated stock prices at each time step


def MC_simulation(S0, K, r, T, N, vol, simulations):
    # MC_simulation: Simulates the price of a European call option using Monte Carlo with multiple time steps in the random walk
    # S0: Initial stock price
    # K: Strike price
    # r: Risk-free interest rate
    # T: Time to maturity (in years)
    # N: Number of time steps in the random walk
    # vol: Volatility of the stock
    # simulations: Number of Monte Carlo simulations to perform
    
    terminal_list = []  # Initialize a list to store the terminal payoffs for each simulation
    
    # Run Monte Carlo simulations
    for i in range(1, simulations+1):
        # Simulate the stock price path using Geometric Brownian Motion with N steps
        ST = geometric_BM_RW_steps(S0, r, T, N, vol).tolist()[-1]  # Get the final (terminal) stock price
        
        # Calculate the payoff for a call option at maturity
        terminal_payoff = call_payoff(ST, K)
        
        # Append the payoff to the terminal_list
        terminal_list.append(terminal_payoff)
    
    # Calculate the average payoff across all simulations
    average_terminal_payoff = sample_mean(terminal_list)
    
    # Discount the average payoff back to the present value using the risk-free rate
    discounted_payoff = average_terminal_payoff * m.exp(-r * T)
    
    return discounted_payoff  # Return the estimated call option price


In [11]:
MC = MC_simulation(29, 28, 0.05, 3/12, 1000, 0.15, 100000)
print("European Call Option Price: £", MC)

European Call Option Price: £ 1.690152328344499
